In [1]:
import tensorflow as tf
import numpy as np
import tqdm
from tqdm import tqdm
import config

In [2]:
gpus = tf.config.list_physical_devices(device_type='GPU')
print(gpus)
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
  except RuntimeError as e:
    print(e)

[]


In [3]:
"""
  Evaluate a given quantized model on a given quantized validation set.
  Eventually save the accuracy of the model in the specified file.

  Args:
    model: tf.lite.Interpreter
      The model to be evaluated.
    valid_X: numpy.ndarray
      The quantized validation set to be used for evaluation.
    valid_Y: numpy.ndarray
      The validation labels set to be used for evaluation.
    model_name: str ["Lenet5"]
      The name of the model to be evaluated.
    quantized_type: str ["int8", "uint8", "int16"]
      The quantization type of the model to be evaluated.
"""
def evaulate_quantized_model_on_quantized_ds(model: tf.lite.Interpreter, 
                                             valid_X: np.ndarray, 
                                             valid_Y: np.ndarray, 
                                             model_name: str, 
                                             quantized_type: str):
  input_details = model.get_input_details()
  output_details = model.get_output_details()

  predictions = []
  desc = model_name + " " + quantized_type + " evaluation"

  for i in tqdm(range(len(valid_X)), desc=desc):
    model.set_tensor(input_details[0]['index'], valid_X[i:i+1])
    model.invoke()
    output_data = model.get_tensor(output_details[0]['index'])
    predictions.append(np.argmax(output_data, axis=1)[0]) 
  predictions = np.array(predictions)
  accuracy = (np.sum(predictions == valid_Y.squeeze())/len(predictions))*100
  
  with open(config.MODELS_ACCURACY_PATH, 'a') as file:
    file.write(model_name + " " + quantized_type + " MNIST[:" + str(len(valid_Y)) + "]: " + str(accuracy) + "\n")

In [5]:
def evaluate_uint8_mnist_x_lenet5():
  model = tf.lite.Interpreter(model_path=config.LENET5_U8_MODEL_PATH)
  model.allocate_tensors()
  valid_X = np.load(config.LENET5_U8_VALIDATION_SET_PREPROCESSED_PATH+".npy")
  valid_Y = np.load(config.VALIDATION_LABELS_PATH)
  evaulate_quantized_model_on_quantized_ds(model, 
                                           valid_X, 
                                           valid_Y, 
                                           'Lenet5', 
                                           'uint8')
  
evaluate_uint8_mnist_x_lenet5()

Lenet5 uint8 evaluation: 100%|██████████| 10000/10000 [00:00<00:00, 17286.90it/s]


In [ ]:
def evaluate_int8_mnist_x_lenet5():
  model = tf.lite.Interpreter(model_path=config.LENET5_I8_MODEL_PATH)
  model.allocate_tensors()
  valid_X = np.load(config.LENET5_I8_VALIDATION_SET_PREPROCESSED_PATH+".npy")
  valid_Y = np.load(config.VALIDATION_LABELS_PATH)
  evaulate_quantized_model_on_quantized_ds(model, 
                                           valid_X, 
                                           valid_Y, 
                                           'Lenet5', 
                                           'int8')

evaluate_int8_mnist_x_lenet5()

Lenet5 int8 evaluation:   0%|          | 0/62 [00:00<?, ?it/s]


ValueError: Cannot set tensor: Got value of type STRING but expected type INT8 for input 0, name: serving_default_input_layer_1:0 